## Pages List (Buy)

In [282]:
import requests
from bs4 import BeautifulSoup

pages_list = []
for i in range(1, 839):
    pages_list.append('https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|'+ str(i) +'||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||')
    #print('Paginator #' + str(i) + ": " + pages_list[i-1])

## Properties Pages List (Buy)

In [283]:
paginator_url = 'https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|1||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')
properties_number = soup.find(id='lblNumInm').get_text()
#print (properties_number + 'properties in total\n')
division = soup.find(id='divAdverts')
properties_pages_list = []
for i in range(0, 34):
    properties = division.find(id='rowIndex_' + str(i))
    property = ''
    if (i == 0):
        property = properties.find('li', class_='media')
        property = property.find('span').attrs.get('onclick')
    else:
        property = properties.find('li', class_='media').attrs.get('onclick')
    url_property = str(property).replace('javascript:window.location=', '')
    url_property = 'https://www.fincaraiz.com.co' + url_property.replace("'", "")
    #print('Property #' + str(i) + ": " + str(url_property))
    properties_pages_list.append(url_property)

## Property Scraping

In [284]:
paginator_url = 'https://www.fincaraiz.com.co/casa-en-venta/medellin/loma_los_bernal-det-4843507.aspx'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Extract Detail Header
detail_header = soup.find('div', class_='row detailheader')
property_price = detail_header.find('div', class_='price').find('h2').get_text()

# Extract Image Content
image_content = soup.find('div', class_='row imageContent')
property_use = image_content.find('div', id='ctl00_phMasterPage_cAdvert_ucMultimedias_icoUsed').get_text()

# Extract Small Features
small_features = soup.find('div', class_='features clearfix')
property_sqrt_meters = small_features.find('span', class_='advertSurface').get_text().split()
property_sqrt_meters = float(" ".join(sqrt_meters)[0:-3].replace(",", "."))
property_rooms = small_features.find('span', class_='advertRooms').get_text().split()
property_rooms = int(" ".join(property_rooms)[-1])
property_baths = small_features.find('span', class_='advertBaths').get_text().split()
property_baths = int(" ".join(property_baths)[-1])
property_park = small_features.find('span', class_='advertGarages').get_text().split()
property_park = int(" ".join(property_park)[-1])

# Estract Big Features
big_features = soup.find('ul', class_='boxcube')
property_admon = ''
property_sector = ''
for li in big_features.find_all('li'):
    value = li.get_text().split()
    value = " ".join(value)
    if(value.find('Área privada') == 0):
        property_private_area = value.replace('Área privada: ', '')[0:-3]
    elif(value.find('Área Const') == 0):
        property_const_area = value.replace('Área Const.: ', '')[0:-3]
    elif(value.find('Precio m²') == 0):
        property_sqrt_meters_price = value.replace('Precio m²: ', '')[0:-3]
    elif(value.find('Estrato') == 0):
        property_stratum = value.replace('Estrato: ', '')
    elif(value.find('Estado') == 0):
        property_feature_status = value.replace('Estado: ', '')
    elif(value.find('Antigüedad') == 0):
        property_antiquity = value.replace('Antigüedad: ', '')
    elif(value.find('Piso No') == 0):
        property_floor = value.replace('Piso No: ', '')[0:-1]
    elif(value.find('Sector') == 0):
        property_sector = value.replace('Sector: ', '')
    elif(value.find('Clima') == 0):
        property_weather = value.replace('Clima: ', '')
    elif(value.find('Admón') == 0):
        property_admon = value.replace('Admón: ', '')

property_features = []
features_object = {
    'squareMeters': property_sqrt_meters,
    'rooms': property_rooms,
    'bathrooms': property_baths,
    'parkingLot': property_parks,
    'privateArea': property_private_area,
    'constructionArea': property_const_area,
    'squareMetersPrice': property_sqrt_meters_price,
    'stratum': property_stratum,
    'status': property_feature_status,
    'antiquity': property_antiquity,
    'floors': property_floor,
    'weather': property_weather,
    'admonPrice': property_admon
}
property_features.append(features_object)

# Extract Description
description = soup.find('div', class_='description')
property_code = int(description.find('span').find('b').get_text())
property_description = description.find('p').text

# Extract Interior Features
interior_features = soup.find('ul', id='tblInitialInteriores')
property_interior_features = [li.text for li in interior_features.find_all('li')]

# Extract Exterior Features
exterior_features = soup.find('ul', id='tblInitialExteriores')
property_exterior_features = [li.text for li in exterior_features.find_all('li')]

# Extract Sector Features
sector_features = soup.find('ul', id='tblInitialdelSector')
property_sector_features = [li.text for li in sector_features.find_all('li')]

## Create JSON

In [285]:
import json

property_dict = {
    'code': property_code,
    'status': 'Venta',
    'use': property_use,
    'price': property_price,
    'sector': property_sector,
    'features': property_features,
    'description': property_description,
    'interiorFeatures': property_interior_features,
    'exteriorFeatures': property_exterior_features,
    'sectorFeatures': property_sector_features
}

print(json.dumps(property_dict, indent=4))

{
    "code": 4843507,
    "status": "Venta",
    "use": "Usado",
    "price": "$ 596.000.000",
    "sector": "Loma de los Bernal",
    "features": [
        {
            "squareMeters": 300.0,
            "rooms": 3,
            "bathrooms": 3,
            "parkingLot": 5,
            "privateArea": "2.088,00",
            "constructionArea": "2.088,00",
            "squareMetersPrice": "285.441",
            "stratum": "Campestre",
            "status": "Excelente",
            "antiquity": "1 a 8 a\u00f1os",
            "floors": "1",
            "weather": "",
            "admonPrice": ""
        }
    ],
    "description": "POR TRASLADO LABORAL, VENTA DIRECTA de moderna casafinca, ubicada en parcelaci\u00f3n campestre en la loma de los Bernal (cerca de AVIVA). Lote 2,088mts con muro perimetral y casa de 1 nivel con 300mts. Incre\u00edble vista a la ciudad,cocina integral,zona de ropas,Sala Comedor, ba\u00f1o social,2 habitaciones con ba\u00f1o y vestier,estudio o 3ra habitaci\u00